In [5]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../../Le_Son_LV2_Project_01')
from confluent_kafka.admin import AdminClient, NewTopic, ConfigResource
from config import load_config
import time

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
kafka_config = load_config(filename="../database.ini", section="local_kafka")
kafka_config

{'servers': 'localhost:9094,localhost:9194,localhost:9294',
 'username': 'kafka',
 'password': 'UnigapKafka@2024',
 'default_consumer_group_id': 'sonhaile-connection',
 'default_consumer_topic': 'product_view',
 'default_producer_topic': 'product_view',
 'default_producer_acks': 'all',
 'default_producer_min_isr': '2'}

In [7]:
# Local Kafka broker config
admin_conf = config = {
            'bootstrap.servers': kafka_config['servers'],
            'security.protocol': 'SASL_PLAINTEXT',       # or 'SASL_SSL' if using SSL
            'sasl.mechanism': 'PLAIN',
            'sasl.username': kafka_config['username'],
            'sasl.password': kafka_config['password']
            }
admin_client = AdminClient(admin_conf)

# --- Create topic ---
topic_name = 'local_test_topic'

new_topic = NewTopic(
    topic=topic_name,
    num_partitions=3,
    replication_factor=3,
    config={
        'retention.ms': '604800000',           # 7 days
        'min.insync.replicas': '2'
    }
)

fs = admin_client.create_topics([new_topic])

# --- Wait for result ---
for topic, f in fs.items():
    try:
        f.result()
        print(f"✅ Topic '{topic}' created successfully")
    except Exception as e:
        print(f"❗ Failed to create topic '{topic}': {e}")

# --- Optional: Verify topic config ---
time.sleep(1)  # wait a bit before describing

resource = ConfigResource(ConfigResource.Type.TOPIC, topic_name)
futures = admin_client.describe_configs([resource])

for res, f in futures.items():
    try:
        configs = f.result()
        print(f"🔍 Topic '{res.name}' configs:")
        for k, v in configs.items():
            print(f"   {k}: {v.value}")
    except Exception as e:
        print(f"❗ Error describing config for topic {res.name}: {e}")


✅ Topic 'local_test_topic' created successfully
🔍 Topic 'local_test_topic' configs:
   compression.type: producer
   leader.replication.throttled.replicas: 
   remote.storage.enable: false
   message.downconversion.enable: true
   min.insync.replicas: 2
   segment.jitter.ms: 0
   local.retention.ms: -2
   cleanup.policy: delete
   flush.ms: 9223372036854775807
   follower.replication.throttled.replicas: 
   segment.bytes: 1073741824
   retention.ms: 604800000
   flush.messages: 9223372036854775807
   message.format.version: 3.0-IV1
   max.compaction.lag.ms: 9223372036854775807
   file.delete.delay.ms: 60000
   max.message.bytes: 1048588
   min.compaction.lag.ms: 0
   message.timestamp.type: CreateTime
   local.retention.bytes: -2
   preallocate: false
   min.cleanable.dirty.ratio: 0.5
   index.interval.bytes: 4096
   unclean.leader.election.enable: false
   retention.bytes: -1
   delete.retention.ms: 86400000
   message.timestamp.after.max.ms: 9223372036854775807
   message.timestamp.b